# Setup

In [1]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

# Question 1

In [2]:
def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    # img is the source image
    # num_horizontal_grid and num_vertical_grid are the number of patches along x and y axes.
    # line_color is the color of the grid line.
    # The output of the function should be image with grids
    img_copy = img.copy()
    
    height, width = img.shape[:2]
    
    M, N = int(height/num_horizontal_grid), int(width/num_vertical_grid)
    x1, y1 = 0, 0

    for y in range(0, height, M):
        for x in range(0, width, N):
    #         if (height - y) < M or (width - x) < N:
    #             break

            y1 = y + M    # lower right coordinate that will be used to construct rectangle
            x1 = x + N

            # Check whether patch lower right coordinate exceeds image height and width
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
    
    cv.imshow('patched image', img_copy)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
img = cv.imread('dog.jfif')
crop_grid(img, 2, 3, (0,255,0))

# Question 2

In [4]:
# get a array of int in the interval of 2
list_int = np.arange(0, 100, 1)
# declare a float array to use as alpha value
list_float = np.zeros(len(list_int))

for i in range(0, len(list_int), 1):
    list_float[i] = list_int[i] / 100

print(list_float)

[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27
 0.28 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41
 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55
 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69
 0.7  0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83
 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97
 0.98 0.99]


In [5]:
img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

# show image multiple in for loop to achieve smooth transition
for i in range(0, len(list_float), 1):
    dst = cv.addWeighted(img1, list_float[i], img2, 0.25, 0)
    cv.imshow('blend_image', dst)
    cv.waitKey(20)

cv.waitKey(0)
cv.destroyAllWindows()

# Question 3

In [6]:
img = cv.imread('lena.jfif')
rows, cols = img.shape[:2]

# Center coordinate
image_center = ((cols-1)/2.0, (rows-1)/2.0)
M = cv.getRotationMatrix2D(image_center, 45, 1)

# Calculate the cosine and sine and take the absolutes of those
abs_cos = abs(M[0,0]) 
abs_sin = abs(M[0,1])

# get the new width and height
new_width = int(rows * abs_sin + cols * abs_cos)
new_height = int(rows * abs_cos + cols * abs_sin)

# Setting the new center coordinate to make sure the image is center
M[0, 2] += new_width/2 - image_center[0]
M[1, 2] += new_height/2 - image_center[1]

dst = cv.warpAffine(img, M, (new_width, new_height))

cv.imshow('rotated_image', dst)
cv.waitKey(0)
cv.destroyAllWindows()

# Question 4

In [7]:
img1 = cv.imread('native-bee.png')
img2 = cv.imread('flower.jfif')

# change flower image from BGR to GRAY for getting the mask and resize to fit the bee image
img2gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
resized_img2 = np.zeros((img1.shape[0], img1.shape[1]), dtype = np.uint8)
resized_img2[35:120, 88:175] = img2gray[35:120, 88:175]

ret, mask = cv.threshold(resized_img2, 70, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask) # inverted mask is used to highlight what part need to be kept in bee image

# get another image in BGR to paste it into bee image
resized_img2_rgb = cv.cvtColor(resized_img2, cv.COLOR_GRAY2BGR)
resized_img2_rgb[35:120, 88:175] = img2[35:120, 88:175]

# black-out the area of the flower in ROI
ROI = cv.bitwise_and(img1, img1, mask = mask_inv)

# Take only region of flower from the image.
flower = cv.bitwise_and(resized_img2_rgb, resized_img2_rgb, mask = mask)

# Put flower in ROI
dst = cv.add(ROI, flower)

cv.imshow('flower_bee', dst)
cv.waitKey(0)
cv.destroyAllWindows()